In [1]:
"""import torch
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo
from torch.utils.checkpoint import *
import torch.nn.functional as F
from torch.nn import init
import numpy as np


class ASPP(nn.Module):
    def __init__(self, C, depth, num_classes, norm=nn.BatchNorm2d, momentum=0.0003, mult=1):
        super(ASPP, self).__init__()
        self._C = C
        self._depth = depth
        self._num_classes = num_classes
        self._norm = norm

        self.global_pooling = nn.AdaptiveAvgPool2d(1)
        self.relu = nn.ReLU(inplace=True)
        self.aspp1 = nn.Conv2d(C, depth, kernel_size=1, stride=1, bias=False)
        self.aspp2 = nn.Conv2d(C, depth, kernel_size=3, stride=1,
                               dilation=int(6*mult), padding=int(6*mult),
                               bias=False)
        self.aspp3 = nn.Conv2d(C, depth, kernel_size=3, stride=1,
                               dilation=int(12*mult), padding=int(12*mult),
                               bias=False)
        self.aspp4 = nn.Conv2d(C, depth, kernel_size=3, stride=1,
                               dilation=int(18*mult), padding=int(18*mult),
                               bias=False)
        self.aspp5 = nn.Conv2d(C, depth, kernel_size=1, stride=1, bias=False)
        self.aspp1_bn = self._norm(depth, momentum)
        self.aspp2_bn = self._norm(depth, momentum)
        self.aspp3_bn = self._norm(depth, momentum)
        self.aspp4_bn = self._norm(depth, momentum)
        self.aspp5_bn = self._norm(depth, momentum)
        self.conv2 = nn.Conv2d(depth * 5, depth, kernel_size=1, stride=1,
                               bias=False)
        self.bn2 = self._norm(depth, momentum)
        self.conv3 = nn.Conv2d(depth, num_classes, kernel_size=1, stride=1)

    def forward(self, x):
        x1 = self.aspp1(x)
        x1 = self.aspp1_bn(x1)
        x1 = self.relu(x1)
        x2 = self.aspp2(x)
        x2 = self.aspp2_bn(x2)
        x2 = self.relu(x2)
        x3 = self.aspp3(x)
        x3 = self.aspp3_bn(x3)
        x3 = self.relu(x3)
        x4 = self.aspp4(x)
        x4 = self.aspp4_bn(x4)
        x4 = self.relu(x4)
        x5 = self.global_pooling(x)
        x5 = self.aspp5(x5)
        x5 = self.aspp5_bn(x5)
        x5 = self.relu(x5)
        x5 = nn.Upsample((x.shape[2], x.shape[3]), mode='bilinear',
                         align_corners=True)(x5)
        x = torch.cat((x1, x2, x3, x4, x5), 1)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)

        return x


class Selayer(nn.Module):

    def __init__(self, inplanes):
        super(Selayer, self).__init__()
        self.global_avgpool = nn.AdaptiveAvgPool2d(1)
        self.conv1 = nn.Conv2d(inplanes, inplanes // 16, kernel_size=1, stride=1)
        self.conv2 = nn.Conv2d(inplanes // 16, inplanes, kernel_size=1, stride=1)
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):

        out = self.global_avgpool(x)

        out = self.conv1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.sigmoid(out)

        return x * out


class BottleneckX(nn.Module):
    expansion = 4
    def __init__(self, inplanes, planes, cardinality, stride=1, downsample=None, dilation=1, norm=None, elastic=False, se=False):
        super(BottleneckX, self).__init__()
        self.se = se
        self.elastic = elastic and stride == 1 and planes < 512
        if self.elastic:
            self.down = nn.AvgPool2d(2, stride=2)
            self.ups = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)
        # half resolution
        self.conv1_d = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1_d = norm(planes)
        self.conv2_d = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, groups=cardinality // 2,
                                 dilation=dilation, padding=dilation, bias=False)
        self.bn2_d = norm(planes)
        self.conv3_d = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        # full resolution
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = norm(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, groups=cardinality // 2,
                               dilation=dilation, padding=dilation, bias=False)
        self.bn2 = norm(planes)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        # after merging
        self.bn3 = norm(planes * self.expansion)
        if self.se:
            self.selayer = Selayer(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.__flops__ = 0

    def forward(self, x):
        residual = x
        out_d = x
        if self.elastic:
            if x.size(2) % 2 > 0 or x.size(3) % 2 > 0:
                out_d = F.pad(out_d, (0, x.size(3) % 2, 0, x.size(2) % 2), mode='replicate')
            out_d = self.down(out_d)

        out_d = self.conv1_d(out_d)
        out_d = self.bn1_d(out_d)
        out_d = self.relu(out_d)

        out_d = self.conv2_d(out_d)
        out_d = self.bn2_d(out_d)
        out_d = self.relu(out_d)

        out_d = self.conv3_d(out_d)

        if self.elastic:
            out_d = self.ups(out_d)
            self.__flops__ += np.prod(out_d[0].shape) * 8
            if out_d.size(2) > x.size(2) or out_d.size(3) > x.size(3):
                out_d = out_d[:, :, :x.size(2), :x.size(3)]

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = out + out_d
        out = self.bn3(out)

        if self.se:
            out = self.selayer(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNext(nn.Module):

    def __init__(self, block, layers, num_classes=1000, seg=False, elastic=False, se=False):
        self.inplanes = 8
        self.cardinality = 32
        self.seg = seg
        self._norm = lambda planes, momentum=0.05 if seg else 0.1: torch.nn.BatchNorm2d(planes, momentum=momentum)

        super(ResNext, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = self._norm(8)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 16, layers[0], elastic=elastic, se=se)
        self.layer2 = self._make_layer(block, 32, layers[1], stride=2, elastic=elastic, se=se)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2, elastic=elastic, se=se)
        if seg:
            self.layer4 = self._make_mg(block, 512, se=se)
            self.aspp = ASPP(512 * block.expansion, 256, num_classes, self._norm)
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                    m.weight.data.normal_(0, math.sqrt(2. / n))
                elif isinstance(m, torch.nn.BatchNorm2d):
                    m.weight.data.fill_(1)
                    m.bias.data.zero_()
        else:
            self.layer4 = self._make_layer(block, 64, layers[3], stride=2, elastic=False, se=se)
            self.avgpool = nn.AdaptiveAvgPool2d(1)
            self.fc = nn.Linear(64 * block.expansion, num_classes)
            init.normal_(self.fc.weight, std=0.01)
            for n, p in self.named_parameters():
                if n.split('.')[-1] == 'weight':
                    if 'conv' in n:
                        init.kaiming_normal_(p, mode='fan_in', nonlinearity='relu')
                    if 'bn' in n:
                        p.data.fill_(1)
                    if 'bn3' in n:
                        p.data.fill_(0)
                elif n.split('.')[-1] == 'bias':
                    p.data.fill_(0)

    def _make_layer(self, block, planes, blocks, stride=1, elastic=False, se=False):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                self._norm(planes * block.expansion),
            )

        layers = list()
        layers.append(block(self.inplanes, planes, self.cardinality, stride, downsample=downsample, norm=self._norm, elastic=elastic, se=se))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, self.cardinality, norm=self._norm, elastic=elastic, se=se))
        return nn.Sequential(*layers)

    def _make_mg(self, block, planes, dilation=2, multi_grid=(1, 2, 4), se=False):
        downsample = nn.Sequential(
            nn.Conv2d(self.inplanes, planes * block.expansion,
                      kernel_size=1, stride=1, dilation=1, bias=False),
            self._norm(planes * block.expansion),
        )

        layers = list()
        layers.append(block(self.inplanes, planes, self.cardinality, downsample=downsample, dilation=dilation*multi_grid[0], norm=self._norm, se=se))
        self.inplanes = planes * block.expansion
        layers.append(block(self.inplanes, planes, self.cardinality, dilation=dilation*multi_grid[1], norm=self._norm, se=se))
        layers.append(block(self.inplanes, planes, self.cardinality, dilation=dilation*multi_grid[2], norm=self._norm, se=se))
        return nn.Sequential(*layers)

    def forward(self, x):
        size = (x.shape[2], x.shape[3])
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        if self.seg:
            for module in self.layer1._modules.values():
                x = checkpoint(module, x)
            for module in self.layer2._modules.values():
                x = checkpoint(module, x)
            for module in self.layer3._modules.values():
                x = checkpoint(module, x)
            for module in self.layer4._modules.values():
                x = checkpoint(module, x)
            x = self.aspp(x)
            x = nn.Upsample(size, mode='bilinear', align_corners=True)(x)
        else:
            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)
            x = self.layer4(x)
            x = self.avgpool(x)
            x = x.view(x.size(0), -1)
            x = self.fc(x)
        return x


def resnext50(seg=False, **kwargs):
    model = ResNext(BottleneckX, [3, 4, 6, 3], seg=seg, elastic=False, **kwargs)
    return model


def se_resnext50(seg=False, **kwargs):
    model = ResNext(BottleneckX, [3, 4, 6, 3], seg=seg, elastic=False, se=True, **kwargs)
    return model


def resnext50_elastic(seg=False, **kwargs):
    model = ResNext(BottleneckX, [6, 8, 5, 3], seg=seg, elastic=True, **kwargs)
    return model


def se_resnext50_elastic(seg=False, **kwargs):
    model = ResNext(BottleneckX, [6, 8, 5, 3], seg=seg, elastic=True, se=True, **kwargs)
    return model


def resnext101(seg=False, **kwargs):
    model = ResNext(BottleneckX, [3, 4, 23, 3], seg=seg, elastic=False, **kwargs)
    return model


def resnext101_elastic(seg=False, **kwargs):
    model = ResNext(BottleneckX, [12, 14, 20, 3], seg=seg, elastic=True, **kwargs)
    return model"""
tempsfsdkshfls = []

In [2]:
import torch
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo
from torch.utils.checkpoint import *
import torch.nn.functional as F
from torch.nn import init
import numpy as np


class ASPP(nn.Module):
    def __init__(self, C, depth, num_classes, norm=nn.BatchNorm2d, momentum=0.0003, mult=1):
        super(ASPP, self).__init__()
        self._C = C
        self._depth = depth
        self._num_classes = num_classes
        self._norm = norm

        self.global_pooling = nn.AdaptiveAvgPool2d(1)
        self.relu = nn.ReLU(inplace=True)
        self.aspp1 = nn.Conv2d(C, depth, kernel_size=1, stride=1, bias=False)
        self.aspp2 = nn.Conv2d(C, depth, kernel_size=3, stride=1,
                               dilation=int(6*mult), padding=int(6*mult),
                               bias=False)
        self.aspp3 = nn.Conv2d(C, depth, kernel_size=3, stride=1,
                               dilation=int(12*mult), padding=int(12*mult),
                               bias=False)
        self.aspp4 = nn.Conv2d(C, depth, kernel_size=3, stride=1,
                               dilation=int(18*mult), padding=int(18*mult),
                               bias=False)
        self.aspp5 = nn.Conv2d(C, depth, kernel_size=1, stride=1, bias=False)
        self.aspp1_bn = self._norm(depth, momentum)
        self.aspp2_bn = self._norm(depth, momentum)
        self.aspp3_bn = self._norm(depth, momentum)
        self.aspp4_bn = self._norm(depth, momentum)
        self.aspp5_bn = self._norm(depth, momentum)
        self.conv2 = nn.Conv2d(depth * 5, depth, kernel_size=1, stride=1,
                               bias=False)
        self.bn2 = self._norm(depth, momentum)
        self.conv3 = nn.Conv2d(depth, num_classes, kernel_size=1, stride=1)

    def forward(self, x):
        x1 = self.aspp1(x)
        x1 = self.aspp1_bn(x1)
        x1 = self.relu(x1)
        x2 = self.aspp2(x)
        x2 = self.aspp2_bn(x2)
        x2 = self.relu(x2)
        x3 = self.aspp3(x)
        x3 = self.aspp3_bn(x3)
        x3 = self.relu(x3)
        x4 = self.aspp4(x)
        x4 = self.aspp4_bn(x4)
        x4 = self.relu(x4)
        x5 = self.global_pooling(x)
        x5 = self.aspp5(x5)
        x5 = self.aspp5_bn(x5)
        x5 = self.relu(x5)
        x5 = nn.Upsample((x.shape[2], x.shape[3]), mode='bilinear',
                         align_corners=True)(x5)
        x = torch.cat((x1, x2, x3, x4, x5), 1)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)

        return x


class Selayer(nn.Module):

    def __init__(self, inplanes):
        super(Selayer, self).__init__()
        self.global_avgpool = nn.AdaptiveAvgPool2d(1)
        self.conv1 = nn.Conv2d(inplanes, inplanes // 16, kernel_size=1, stride=1)
        self.conv2 = nn.Conv2d(inplanes // 16, inplanes, kernel_size=1, stride=1)
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):

        out = self.global_avgpool(x)

        out = self.conv1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.sigmoid(out)

        return x * out


class BottleneckX(nn.Module):
    expansion = 4
    def __init__(self, inplanes, planes, cardinality, stride=1, downsample=None, dilation=1, norm=None, elastic=False, se=False):
        super(BottleneckX, self).__init__()
        self.se = se
        self.elastic = elastic and stride == 1 and planes < 512
        if self.elastic:
            self.down = nn.AvgPool2d(2, stride=2)
            self.ups = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)
        # half resolution
        self.conv1_d = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1_d = norm(planes)
        self.conv2_d = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, groups=cardinality // 2,
                                 dilation=dilation, padding=dilation, bias=False)
        self.bn2_d = norm(planes)
        self.conv3_d = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        # full resolution
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = norm(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, groups=cardinality // 2,
                               dilation=dilation, padding=dilation, bias=False)
        self.bn2 = norm(planes)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        # after merging
        self.bn3 = norm(planes * self.expansion)
        if self.se:
            self.selayer = Selayer(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.__flops__ = 0

    def forward(self, x):
        residual = x
        out_d = x
        if self.elastic:
            if x.size(2) % 2 > 0 or x.size(3) % 2 > 0:
                out_d = F.pad(out_d, (0, x.size(3) % 2, 0, x.size(2) % 2), mode='replicate')
            out_d = self.down(out_d)

        out_d = self.conv1_d(out_d)
        out_d = self.bn1_d(out_d)
        out_d = self.relu(out_d)

        out_d = self.conv2_d(out_d)
        out_d = self.bn2_d(out_d)
        out_d = self.relu(out_d)

        out_d = self.conv3_d(out_d)

        if self.elastic:
            out_d = self.ups(out_d)
            self.__flops__ += np.prod(out_d[0].shape) * 8
            if out_d.size(2) > x.size(2) or out_d.size(3) > x.size(3):
                out_d = out_d[:, :, :x.size(2), :x.size(3)]

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = out + out_d
        out = self.bn3(out)

        if self.se:
            out = self.selayer(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNext(nn.Module):

    def __init__(self, block, layers, num_classes=1000, seg=False, elastic=False, se=False):
        self.inplanes = 64
        self.cardinality = 32
        self.seg = seg
        self._norm = lambda planes, momentum=0.05 if seg else 0.1: torch.nn.BatchNorm2d(planes, momentum=momentum)

        super(ResNext, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = self._norm(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], elastic=elastic, se=se)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, elastic=elastic, se=se)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, elastic=elastic, se=se)
        if seg:
            self.layer4 = self._make_mg(block, 512, se=se)
            self.aspp = ASPP(512 * block.expansion, 256, num_classes, self._norm)
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                    m.weight.data.normal_(0, math.sqrt(2. / n))
                elif isinstance(m, torch.nn.BatchNorm2d):
                    m.weight.data.fill_(1)
                    m.bias.data.zero_()
        else:
            self.layer4 = self._make_layer(block, 512, layers[3], stride=2, elastic=False, se=se)
            self.avgpool = nn.AdaptiveAvgPool2d(1)
            self.fc = nn.Linear(512 * block.expansion, num_classes)
            init.normal_(self.fc.weight, std=0.01)
            for n, p in self.named_parameters():
                if n.split('.')[-1] == 'weight':
                    if 'conv' in n:
                        init.kaiming_normal_(p, mode='fan_in', nonlinearity='relu')
                    if 'bn' in n:
                        p.data.fill_(1)
                    if 'bn3' in n:
                        p.data.fill_(0)
                elif n.split('.')[-1] == 'bias':
                    p.data.fill_(0)

    def _make_layer(self, block, planes, blocks, stride=1, elastic=False, se=False):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                self._norm(planes * block.expansion),
            )

        layers = list()
        layers.append(block(self.inplanes, planes, self.cardinality, stride, downsample=downsample, norm=self._norm, elastic=elastic, se=se))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, self.cardinality, norm=self._norm, elastic=elastic, se=se))
        return nn.Sequential(*layers)

    def _make_mg(self, block, planes, dilation=2, multi_grid=(1, 2, 4), se=False):
        downsample = nn.Sequential(
            nn.Conv2d(self.inplanes, planes * block.expansion,
                      kernel_size=1, stride=1, dilation=1, bias=False),
            self._norm(planes * block.expansion),
        )

        layers = list()
        layers.append(block(self.inplanes, planes, self.cardinality, downsample=downsample, dilation=dilation*multi_grid[0], norm=self._norm, se=se))
        self.inplanes = planes * block.expansion
        layers.append(block(self.inplanes, planes, self.cardinality, dilation=dilation*multi_grid[1], norm=self._norm, se=se))
        layers.append(block(self.inplanes, planes, self.cardinality, dilation=dilation*multi_grid[2], norm=self._norm, se=se))
        return nn.Sequential(*layers)

    def forward(self, x):
        size = (x.shape[2], x.shape[3])
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        if self.seg:
            for module in self.layer1._modules.values():
                x = checkpoint(module, x)
            for module in self.layer2._modules.values():
                x = checkpoint(module, x)
            for module in self.layer3._modules.values():
                x = checkpoint(module, x)
            for module in self.layer4._modules.values():
                x = checkpoint(module, x)
            x = self.aspp(x)
            x = nn.Upsample(size, mode='bilinear', align_corners=True)(x)
        else:
            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)
            x = self.layer4(x)
            x = self.avgpool(x)
            x = x.view(x.size(0), -1)
            x = self.fc(x)
        return x


def resnext50(seg=False, **kwargs):
    model = ResNext(BottleneckX, [3, 4, 6, 3], seg=seg, elastic=False, **kwargs)
    return model


def se_resnext50(seg=False, **kwargs):
    model = ResNext(BottleneckX, [3, 4, 6, 3], seg=seg, elastic=False, se=True, **kwargs)
    return model


def resnext50_elastic(seg=False, **kwargs):
    model = ResNext(BottleneckX, [6, 8, 5, 3], seg=seg, elastic=True, **kwargs)
    return model


def se_resnext50_elastic(seg=False, **kwargs):
    model = ResNext(BottleneckX, [6, 8, 5, 3], seg=seg, elastic=True, se=True, **kwargs)
    return model


def resnext101(seg=False, **kwargs):
    model = ResNext(BottleneckX, [3, 4, 23, 3], seg=seg, elastic=False, **kwargs)
    return model


def resnext101_elastic(seg=False, **kwargs):
    model = ResNext(BottleneckX, [12, 14, 20, 3], seg=seg, elastic=True, **kwargs)
    return model

In [3]:
INTERNAL_DATA_PATH = 'mnist_4x/'
INTERNAL_DATA_PATH_MAIN = 'mnist_4x/resized'

In [4]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision
import torch
from torch.utils.data import Dataset, DataLoader
import os
import matplotlib.pyplot as plt
import numpy as np


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
transform = transforms.Compose(
    [
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform1 = transforms.Compose(
    [
    transforms.RandomAffine(0, scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])



In [7]:
trainMnistDataset = datasets.ImageFolder(root=os.path.join(INTERNAL_DATA_PATH_MAIN,'training'),transform = transform1)
testMnistDataset = datasets.ImageFolder(root=os.path.join(INTERNAL_DATA_PATH_MAIN,'testing'),transform = transform)

trainloader = torch.utils.data.DataLoader(trainMnistDataset,
                                          batch_size = 32, 
                                          shuffle=True,
                                         num_workers=8)

testloader = torch.utils.data.DataLoader(testMnistDataset,
                                          batch_size = 32, 
                                          shuffle=False,
                                        num_workers=8)

In [8]:
'''for i,image in enumerate(trainloader):
    print(image[0])
    print(image[1])
    break'''

'for i,image in enumerate(trainloader):\n    print(image[0])\n    print(image[1])\n    break'

In [9]:
from parallel import DataParallelModel,DataParallelCriterion
from torch.nn.parallel.data_parallel import DataParallel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnext50_elastic(num_classes=10)
if torch.cuda.device_count() > 1:
    model = DataParallel(model)
model.to(device)

DataParallel(
  (module): ResNext(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BottleneckX(
        (down): AvgPool2d(kernel_size=2, stride=2, padding=0)
        (ups): Upsample(scale_factor=2.0, mode=bilinear)
        (conv1_d): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1_d): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2_d): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
        (bn2_d): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3_d): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (conv1): Conv2d(64, 6

In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
#criterion = DataParallelCriterion(criterion,device_ids=[0,1])
optimizer = optim.SGD(model.parameters(), lr=0.1, 
                    momentum=0.9)

In [11]:
import torch.optim as optim
from tqdm import tqdm
import sys

trn_loss_list = []
val_loss_list = []
num_batches = len(trainloader)
for epoch in range(30):
    running_loss = 0.0
    for i,image in tqdm(enumerate(trainloader)):
        #data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        c1 = image[0].to(device)
        c6 = image[1].to(device)
        output = model(c1)
        #output = torch.nn.parallel.gather(output,'cuda:1')
        loss = criterion(output, c6)
        loss.backward()    # calc gradients
        optimizer.step()   # update gradients
        running_loss += loss.item()
        #print(i)
        
        #if (i+1) % 600 == 0 :
    with torch.no_grad(): # very very very very important!!!
        val_loss = 0.0
        class_correct = list(0. for i in range(10))
        class_total = list(0. for i in range(10))
        correct = 0
        total = 0
        for j,val in enumerate(testloader):
            v1 = val[0].to(device)
            val_labels = val[1].to(device)
            val_output = model(v1)
            v_loss = criterion(val_output, val_labels)
            val_loss += v_loss
            _, predicted = torch.max(val_output, 1)
            c = (predicted == val_labels).squeeze()
            total += val_labels.size(0)
            correct += (predicted == val_labels).sum().item()
            for i in range(len(val_labels)):
                val_label = val_labels[i]
                class_correct[val_label] += c[i].item()
                class_total[val_label] += 1

        for i in range(10):
            if class_total[i]==0:
                print('class_total = 0',class_correct,class_total)
            else:
                print('Accuracy of %5s : %2d %%' % (i, 100 * class_correct[i] / class_total[i]))

    print("epoch: {}/{} | step: {}/{} | trn loss: {:.4f} | val loss: {:.4f}".format(
        epoch+1, 30, i+1, num_batches, running_loss / 600, val_loss / len(testloader)
    ))        
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

    trn_loss_list.append(running_loss/600)
    val_loss_list.append(val_loss/len(testloader))
    running_loss = 0.0

        
    #optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    

1875it [06:23,  4.89it/s]


Accuracy of     0 : 78 %
Accuracy of     1 : 32 %
Accuracy of     2 : 37 %
Accuracy of     3 : 57 %
Accuracy of     4 : 31 %
Accuracy of     5 : 41 %
Accuracy of     6 : 67 %
Accuracy of     7 : 71 %
Accuracy of     8 : 56 %
Accuracy of     9 : 72 %
epoch: 1/30 | step: 10/1875 | trn loss: 1.0417 | val loss: 2.1910
Accuracy of the network on the 10000 test images: 54 %


1875it [06:21,  4.91it/s]


Accuracy of     0 : 84 %
Accuracy of     1 : 29 %
Accuracy of     2 : 67 %
Accuracy of     3 : 47 %
Accuracy of     4 : 31 %
Accuracy of     5 : 77 %
Accuracy of     6 : 59 %
Accuracy of     7 : 84 %
Accuracy of     8 : 78 %
Accuracy of     9 : 59 %
epoch: 2/30 | step: 10/1875 | trn loss: 0.1624 | val loss: 1.9212
Accuracy of the network on the 10000 test images: 61 %


1875it [06:23,  4.89it/s]


...

Accuracy of     0 : 93 %
Accuracy of     1 : 66 %
Accuracy of     2 : 87 %
Accuracy of     3 : 74 %
Accuracy of     4 : 74 %
Accuracy of     5 : 91 %
Accuracy of     6 : 93 %
Accuracy of     7 : 88 %
Accuracy of     8 : 90 %
Accuracy of     9 : 85 %
epoch: 28/30 | step: 10/1875 | trn loss: 0.0086 | val loss: 0.9591
Accuracy of the network on the 10000 test images: 84 %


1875it [06:22,  4.90it/s]


Accuracy of     0 : 93 %
Accuracy of     1 : 62 %
Accuracy of     2 : 87 %
Accuracy of     3 : 78 %
Accuracy of     4 : 85 %
Accuracy of     5 : 91 %
Accuracy of     6 : 93 %
Accuracy of     7 : 90 %
Accuracy of     8 : 93 %
Accuracy of     9 : 87 %
epoch: 29/30 | step: 10/1875 | trn loss: 0.0078 | val loss: 0.8509
Accuracy of the network on the 10000 test images: 85 %


1875it [06:21,  4.91it/s]


Accuracy of     0 : 97 %
Accuracy of     1 : 58 %
Accuracy of     2 : 89 %
Accuracy of     3 : 81 %
Accuracy of     4 : 80 %
Accuracy of     5 : 90 %
Accuracy of     6 : 93 %
Accuracy of     7 : 86 %
Accuracy of     8 : 91 %
Accuracy of     9 : 89 %
epoch: 30/30 | step: 10/1875 | trn loss: 0.0087 | val loss: 0.8534
Accuracy of the network on the 10000 test images: 85 %


In [12]:
val_gen_arr = []
for i in range(1,113):
    transform = transforms.Compose(
        [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    testMnistDataset1 = datasets.ImageFolder(root=os.path.join('mnist_sized',str(i)),transform = transform)

    testloader1 = torch.utils.data.DataLoader(testMnistDataset1,
                                              batch_size = 32, 
                                              shuffle=False,
                                            num_workers=8)
    val_gen_arr.append(testloader1)
    

In [16]:
accuracy = []

for a in tqdm(range(len(val_gen_arr))):
    testloader = val_gen_arr[a]
    correct = 0
    total = 0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for image in testloader:
            c1 = image[0].to(device)
            val_labels = image[1].to(device)
            outputs = model(c1)
            _, predicted = torch.max(outputs.data, 1)
            c = (predicted == val_labels).squeeze()
            total += val_labels.size(0)
            correct += (predicted == val_labels).sum().item()
            for i in range(len(val_labels)):
                val_label = val_labels[i]
                class_correct[val_label] += c[i].item()
                class_total[val_label] += 1
        
        print("IMAGE SIZE :",str(a+1))
        for k in range(10):
            if class_total[k]==0:
                print('class_total = 0',class_correct,class_total)
            else:
                print('Accuracy of %5s : %2d %%' % (k, 100 * class_correct[k] / class_total[k]))
            #print(total,correct,end='')

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
    accuracy.append((100 * correct / total))

  1%|          | 1/112 [00:03<06:14,  3.37s/it]

IMAGE SIZE : 1
Accuracy of     0 :  0 %
Accuracy of     1 :  0 %
Accuracy of     2 :  0 %
Accuracy of     3 :  0 %
Accuracy of     4 : 78 %
Accuracy of     5 :  0 %
Accuracy of     6 :  0 %
Accuracy of     7 :  0 %
Accuracy of     8 :  0 %
Accuracy of     9 :  1 %
Accuracy of the network on the 10000 test images: 7 %


  2%|▏         | 2/112 [00:06<06:10,  3.37s/it]

IMAGE SIZE : 2
Accuracy of     0 :  0 %
Accuracy of     1 : 16 %
Accuracy of     2 :  1 %
Accuracy of     3 :  0 %
Accuracy of     4 : 70 %
Accuracy of     5 :  0 %
Accuracy of     6 :  0 %
Accuracy of     7 :  0 %
Accuracy of     8 :  0 %
Accuracy of     9 :  0 %
Accuracy of the network on the 10000 test images: 8 %


  3%|▎         | 3/112 [00:10<06:06,  3.36s/it]

...

IMAGE SIZE : 110
Accuracy of     0 : 98 %
Accuracy of     1 : 66 %
Accuracy of     2 : 93 %
Accuracy of     3 : 83 %
Accuracy of     4 : 80 %
Accuracy of     5 : 95 %
Accuracy of     6 : 95 %
Accuracy of     7 : 88 %
Accuracy of     8 : 92 %
Accuracy of     9 : 87 %
Accuracy of the network on the 10000 test images: 87 %


 99%|█████████▉| 111/112 [06:12<00:03,  3.35s/it]

IMAGE SIZE : 111
Accuracy of     0 : 98 %
Accuracy of     1 : 65 %
Accuracy of     2 : 92 %
Accuracy of     3 : 83 %
Accuracy of     4 : 80 %
Accuracy of     5 : 95 %
Accuracy of     6 : 95 %
Accuracy of     7 : 86 %
Accuracy of     8 : 92 %
Accuracy of     9 : 87 %
Accuracy of the network on the 10000 test images: 87 %


100%|██████████| 112/112 [06:15<00:00,  3.35s/it]

IMAGE SIZE : 112
Accuracy of     0 : 98 %
Accuracy of     1 : 68 %
Accuracy of     2 : 92 %
Accuracy of     3 : 83 %
Accuracy of     4 : 80 %
Accuracy of     5 : 94 %
Accuracy of     6 : 95 %
Accuracy of     7 : 85 %
Accuracy of     8 : 92 %
Accuracy of     9 : 87 %
Accuracy of the network on the 10000 test images: 87 %


In [18]:
for i in range(len(accuracy)):
    print(accuracy[i],',',end='')

7.9 ,8.7 ,10.4 ,9.0 ,9.2 ,8.1 ,9.0 ,8.7 ,8.1 ,7.6 ,8.0 ,7.7 ,9.0 ,8.7 ,8.7 ,8.6 ,8.0 ,9.1 ,8.6 ,9.8 ,9.3 ,11.2 ,10.7 ,12.1 ,12.6 ,13.6 ,13.9 ,15.3 ,15.3 ,17.1 ,16.5 ,18.9 ,18.9 ,20.9 ,20.2 ,22.5 ,23.4 ,25.2 ,26.6 ,29.6 ,30.0 ,32.7 ,33.0 ,35.3 ,36.2 ,37.9 ,39.2 ,40.5 ,41.7 ,43.7 ,44.5 ,46.6 ,47.1 ,50.0 ,50.9 ,52.9 ,53.6 ,55.9 ,56.9 ,59.5 ,60.3 ,63.0 ,63.5 ,65.8 ,66.8 ,69.4 ,69.3 ,71.3 ,72.4 ,74.7 ,73.9 ,75.9 ,75.9 ,78.1 ,78.2 ,79.8 ,79.3 ,81.3 ,81.4 ,82.8 ,82.9 ,84.0 ,83.7 ,84.2 ,84.5 ,84.8 ,85.2 ,85.3 ,85.5 ,86.2 ,85.9 ,86.5 ,86.5 ,86.7 ,86.2 ,86.9 ,86.7 ,86.7 ,86.7 ,86.7 ,86.8 ,87.3 ,87.0 ,87.5 ,87.0 ,87.6 ,87.3 ,87.4 ,87.3 ,87.7 ,87.3 ,87.4 ,

In [14]:
for images in val_gen_arr[1]:
    print(images)
    break

[tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],

         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],

         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1

In [15]:
with torch.no_grad(): # very very very very important!!!
    val_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    correct = 0
    total = 0
    for j,val in enumerate(testloader):
        v1 = val[0].to(device)
        val_labels = val[1].to(device)
        val_output = model(v1)
        v_loss = criterion(val_output, val_labels)
        val_loss += v_loss
        _, predicted = torch.max(val_output, 1)
        c = (predicted == val_labels).squeeze()
        total += val_labels.size(0)
        correct += (predicted == val_labels).sum().item()
        for i in range(len(val_labels)):
            val_label = val_labels[i]
            class_correct[val_label] += c[i].item()
            class_total[val_label] += 1

    for i in range(10):
        if class_total[i]==0:
            print('class_total = 0',class_correct,class_total)
        else:
            print('Accuracy of %5s : %2d %%' % (i, 100 * class_correct[i] / class_total[i]))

print("epoch: {}/{} | step: {}/{} | trn loss: {:.4f} | val loss: {:.4f}".format(
    epoch+1, 30, i+1, num_batches, running_loss / 200, val_loss / len(testloader)
))        
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of     0 : 98 %
Accuracy of     1 : 65 %
Accuracy of     2 : 92 %
Accuracy of     3 : 83 %
Accuracy of     4 : 80 %
Accuracy of     5 : 95 %
Accuracy of     6 : 95 %
Accuracy of     7 : 86 %
Accuracy of     8 : 92 %
Accuracy of     9 : 87 %
epoch: 30/30 | step: 10/1875 | trn loss: 0.0000 | val loss: 0.7342
Accuracy of the network on the 10000 test images: 87 %
